# Using VGG16 for our CSK team Classifier Classifer

### Loading the MobileNet Model

Freeze all layers except the top 4, as we'll only be training the top 4

In [3]:
from keras.applications import VGG16

# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the MobileNet model without the top or FC layers
VGG = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in VGG.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(VGG.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


### Let's make a function that returns our FC Head

In [4]:
def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

### Let's add our FC Head back onto MobileNet

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes = 10

FC_Head = lw(VGG, num_classes)

model = Model(inputs = VGG.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

### Loading our dataset

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'Csk_team_Classification/train/'
validation_data_dir = 'Csk_team_Classification/validation/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 20
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 27 images belonging to 10 classes.
Found 39 images belonging to 10 classes.


### Training out Model
- Note we're using checkpointing and early stopping

In [9]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("Csk_team_Classification.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 27
nb_validation_samples = 39

# We only train 5 EPOCHS 
epochs = 5
batch_size = 2

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
13/13 [==============================] - 83s 6s/step - loss: 2.3040 - accuracy: 0.2367 - val_loss: 2.6718 - val_accuracy: 0.0997

Epoch 00001: val_loss improved from inf to 2.67184, saving model to Csk_team_Classification.h5
Epoch 2/5
13/13 [==============================] - 93s 7s/step - loss: 1.9308 - accuracy: 0.2802 - val_loss: 2.5259 - val_accuracy: 0.1595

Epoch 00002: val_loss improved from 2.67184 to 2.52588, saving model to Csk_team_Classification.h5
Epoch 3/5
13/13 [==============================] - 100s 8s/step - loss: 1.5303 - accuracy: 0.4438 - val_loss: 2.3279 - val_accuracy: 0.2049

Epoch 00003: val_loss improved from 2.52588 to 2.32792, saving model to Csk_team_Classification.h5
Epoch 4/5
13/13 [==============================] - 110s 8s/step - loss: 1.3039 - accuracy: 0.5659 - val_loss: 3.1916 - val_accuracy: 0.1514

Epoch 00004: val_loss did not improve from 2.32792
Epoch 5/5
13/13 [==============================] - 109s 8s/step - loss: 0.9313 - accuracy: 0.6

In [ ]:
model.summary()

### Loading our classifer


In [10]:
from keras.models import load_model

classifier = load_model('Csk_team_Classification.h5')

### Testing our classifer on some test images

In [11]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

dictn = {"[0]": "Albie Morkel " , 
        "[1]":  "Deepak Chahar",
        "[2]": "DJ Bravo",
        "[3]": "FAF Du Plesis",
        "[4]": "Imran Tahir",
        "[5]": "MS Dhoni",
        "[6]": "Ravindra Jadeja",
        "[7]": "Sam Bilings",
        "[8]": "Shane Watson ",
        "[9]": "SK Raina"}

dict_n = {"Albie Morkel ": "Albie Morkel",
          "Deepak Chahar": "Deepak Chahar",
          "DJ Bravo": "DJ Bravo",
          "FAF Du Plesis": "FAF Du Plessis",
          "Imran Tahir": "Imran Tahir",
          "MS Dhoni": "MS Dhoni",
          "Ravindra Jadeja": "Ravindra Jadeja",
          "Sam Bilings": "Sam Billings",
          "Shane Watson": "Shane Watson ",
          "SK Raina": "SK Raina"}
                      

def draw_test(name, pred, im):
    team = dictn[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, team, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("Csk_team_Classification/train/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - Shane Watson 
Class - Sam Billings
Class - Deepak Chahar
Class - MS Dhoni
Class - DJ Bravo
Class - Ravindra Jadeja


KeyError: 'Albie Morkel'